# tools

> Fill in a module description here

In [ ]:
#| default_exp tools

In [ ]:
#| export
import os, re
import json

from lisette import *

from toolslm.funccall import mk_ns, call_func
from toolslm.shell import get_shell

from litellm import completion
from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
#| export
def prep_shell(context, model="openai/openai/gpt-oss-120b", base_url=None):
    """
    Prepare a REPL shell environment for RLM with context and recursive LLM query capability.
    
    Args:
        context: The text/data to be analyzed (stored as a variable in the shell)
        model: Model name (OpenAI-compatible)
        base_url: API base URL for your LLM gateway
    
    Returns:
        IPython shell instance with `context` and `llm_query()` available
    """
    def llm_query(query: str):
        "Call an LLM with the given query and return the answer"
        kwargs = {"model": model, "messages": [{"role": "user", "content": query}]}
        if base_url:
            kwargs["api_base"] = base_url
        response = completion(**kwargs)
        return response.choices[0].message.content

    sh = get_shell()
    sh.user_ns['context'] = context
    sh.user_ns['llm_query'] = llm_query
    return sh

In [ ]:
#| export
def make_run_repl(sh, max_output=5000):
    """
    Create a run_repl tool function that executes Python code in the given shell.
    
    Args:
        sh: IPython shell instance (from prep_shell)
        max_output: Maximum characters to return from output (default: 5000)
    
    Returns:
        Function that executes code and returns output (truncated to max_output chars)
    """
    def run_repl(python_code: str) -> str:
        "Execute Python code in the REPL environment"
        result = sh.run_cell(python_code)
        if result.error_in_exec or result.error_before_exec:
            return f"Error: {result.error_in_exec or result.error_before_exec}"
        output = result.stdout or ""
        if result.result is not None:
            output += str(result.result)
        return output[:max_output] if max_output else output
    return run_repl

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()